In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [ ]:
#Reading the training data set
DATADIR = "/content/drive/My Drive/Colab Notebooks/multiclass-image-competition/train/"
CATEGORIES = ["1-Tomato 3", "2-Tomato 4", "3-Tomato Cherry Red", "4-Tomato Maroon", "5-Tomato Yellow", "6-Walnut"]
!ls /content/drive/My\ Drive/Colab\ Notebooks

**Creating Training Data**

In [69]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random

IMG_SIZE = 100

training_data = []
def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) #path to train data directory
        class_num = CATEGORIES.index(category) #categorizing the images
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
               # plt.imshow(new_array)
                #plt.show()
                training_data.append([new_array,class_num])
            except Exception as e: #broken images
                pass

create_training_data()
print(len(training_data)) #total images

random.shuffle(training_data) 

#for sample in training_data:
#   print(sample[1])

X=[]
Y=[]
for features, label in training_data:
    X.append(features)
    Y.append(label)
    
X = np.array(X) #-1 is how many features we have, shape should be imagesize x imagesize, 1 because it is in grayscale
Y = np.array(Y)
X = X/255.0
#print (X)

2330


***CNN Architecture***

*model fitting*

In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pickle

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("X.shape=",X.shape)
print("x_train.shape=",x_train.shape)
print("y_train.shape=",y_train.shape)
print("x_test.shape=",x_test.shape)
print("y_test.shape=",y_test.shape)

callback = EarlyStopping(monitor='val_loss', patience=5)
# This callback will stop the training when there is no improvement in
# the validation loss for five consecutive epochs.

model = Sequential()

model.add(Conv2D(14,(3,3),input_shape=X.shape[1:]))
#model.add(Conv2D(7,(3,3),input_shape=x_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(14,(3,3)))
#model.add(Conv2D(7,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dropout(0.2))
model.add(Dense(6))
model.add(Activation('softmax'))


model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=['accuracy']) #we now have 6 cateogories which is my we will perform categorical classification
model.fit(x_train, y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test),verbose=0)


X.shape= (2330, 100, 100, 3)
x_train.shape= (1864, 100, 100, 3)
y_train.shape= (1864,)
x_test.shape= (466, 100, 100, 3)
y_test.shape= (466,)


**Testing the Predictions and adding them to the CSV file**

In [79]:
#reading the test dataset
DATADIRtest = "/content/drive/My Drive/Colab Notebooks/multiclass-image-competition/test"

filename = "/content/drive/My Drive/Colab Notebooks/predictions.csv"
File=open(filename, "w")
File.write("filename,category") #header name
File.write("\n")

for img in os.listdir(DATADIRtest):
    try:
        img_array = cv2.imread(os.path.join(DATADIRtest,img), cv2.IMREAD_COLOR)
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        #plt.imshow(new_array)
        #plt.show()
        new_array = np.array(new_array).reshape(-1, IMG_SIZE, IMG_SIZE,3) #-1 is how many features we have, shape should be imagesize x imagesize
        pred = model.predict(new_array) #prediction model
        #print(pred)
        
        File.write(img + "," + str(np.argmax(pred)+1)) #categories range from 1 to 6 
        File.write("\n")
    except Exception as e: #broken images
        pass


File.close()

[[0. 0. 0. 0. 1. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 0. 1. 0.]]
[[0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 1. 0. 0.]]
[[0. 0. 0. 0. 1. 0.]]
[[1. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 1. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 1. 0. 0.]]
[[0. 1. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 1. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 1. 0. 0.]]
[[1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0.]]
[[1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 0. 1. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 1. 0. 0.]]
[[0. 0. 0. 0. 1. 0.]]
[[0. 0. 0. 0. 1. 0.]]
[[0. 0. 0. 1. 0. 0.]]
[[0. 0. 0. 0. 0. 1.]]
[[1. 0. 0. 0. 0. 0.]]
[[0. 1. 0.